# cr3 CTF 2024
##   guessing-revenge  | 417 pts

Task description:

```
I am very disappointed, please try this one :sob:

$ nc 1337.sb 20004

Author: kibastian, es3n1n
```

Attachments: [main.sage](./main.sage)

The most interesting part is this function:

In [44]:
def submit_b(B):
    if not B.is_invertible():
        return "non invertible"

    n = B.nrows()
    if n != FLAG_N:
        return "rows"

    diagonal_element = B[0, 0]

    identity_multiple = Matrix(F, identity_matrix(n))
    for i in range(n):
        identity_multiple[i, i] = diagonal_element

    if B == identity_multiple:
        return "identity_multiple"

    return B * A * (B ^ -1) * (A ^ -1) * FLAG


print("aight, lets see what you got")
while True:
    try:
        matrix = deserialize_mtx(input())
        res = submit_b(matrix)
        if isinstance(res, str):
            print("https://youtu.be/G7b4VMEATNo?t=18")
            break
    except Exception as e:
        print("https://youtu.be/G7b4VMEATNo?t=19")
        break

    print(serialize_mtx(res))

## Solution

They fixed the bug and now identity matrix check is consistent.

We need to somehow recover flag from the equation:

$B * A * B^{-1} * A^{-1} * \vec{flag}$, where $B$ should be non identity. 

First guess is permutations.

Let's look at permutations of order 3 on 3x3 matrices:

\begin{align*}
A &= \begin{pmatrix}
a_{00} & a_{01} & a_{02} \\
a_{10} & a_{11} & a_{12} \\
a_{20} & a_{21} & a_{22} \\
\end{pmatrix}
\end{align*}

\begin{align*}
A^{-1} &= \begin{pmatrix}
b_{00} & b_{01} & b_{02} \\
b_{10} & b_{11} & b_{12} \\
b_{20} & b_{21} & b_{22} \\
\end{pmatrix}, \\
\end{align*}

We have 6 permutations on 3 elements, namely:

(1, 2, 3)

(1, 3, 2)

(2, 1, 3)

(2, 3, 1)

(3, 1, 2)

(3, 2, 1)

First one is an identity so we don't need it.

We're left with 5 of them, and let's forget for a moment about the leftmost matrix, since we can easily erase it in the end by multiplying with it's inverse.

We know that right permutation matrix permutes the columns of matrix $A$

Let's take a look at 00 element of the resulting product($A * P * A^{-1}$) using each of 5 permutations, noting that we need to somehow obtain $a_{00} * b_{00} + a_{01} * b_{10} + a_{02} * b_{20} = 1$:

(1, 3, 2):

$r_1 = a_{00}*b_{00} + a_{02}*b_{10} + a_{01}*b_{20}$

(2, 1, 3)

$r_2 = a_{01}*b_{00} + a_{00}*b_{10} + a_{02}*b_{20}$

(2, 3, 1)

$r_3 = a_{02}*b_{00} + a_{00}*b_{10} + a_{01}*b_{20}$

(3, 1, 2)

$r_4 = a_{01}*b_{00} + a_{02}*b_{10} + a_{00}*b_{20}$

(3, 2, 1)

$r_5 = a_{02}*b_{00} + a_{01}*b_{10} + a_{00}*b_{20}$

Now look at $r_1 + r_2 + r_5 - r_3 - r_4$ (It looked better on paper...)

\begin{aligned}
& (a_{00} \cdot b_{00} + a_{02} \cdot b_{10} + a_{01} \cdot b_{20}) + \\
& (a_{01} \cdot b_{00} + a_{00} \cdot b_{10} + a_{02} \cdot b_{20}) + \\
& (a_{02} \cdot b_{00} + a_{01} \cdot b_{10} + a_{00} \cdot b_{20}) - \\
& (a_{02} \cdot b_{00} + a_{00} \cdot b_{10} + a_{01} \cdot b_{20}) - \\
& (a_{01} \cdot b_{00} + a_{02} \cdot b_{10} + a_{00} \cdot b_{20}) \\
&= a_{00} \cdot b_{00} + a_{02} \cdot b_{20} + a_{01} \cdot b_{10} \\
&= 1
\end{aligned}

And we end up with identity.

As simple as that we can use these 3x3 matrices inside the 55x55 matrix.

In [3]:
p = (2 ** 31) - 1
F = GF(p)
FLAG_N = 55

def serialize_mtx(mtx):
    rows = []
    for row in mtx.rows():
        rows.append(','.join(str(elem) for elem in row))
    return '[' + ';'.join(rows) + ']'


def deserialize_mtx(mtx):
    print(mtx)
    rows = mtx.strip('[]\n').split(';')
    rows = [[int(num) for num in row.split(',')] for row in rows]
    return Matrix(F, rows)

In [4]:
from itertools import permutations


def get_permutations():
    perms = []
    for x in permutations([[1, 0, 0], [0, 1, 0], [0, 0, 1]]):
        perms.append(Matrix(F, x))
    return perms[1:]
perms = get_permutations()

In [5]:
from pwn import remote, context
context.log_level = "error"

host, port = "1337.sb",  20004
r = remote(host,port)

# handle pow
print(r.recvline()) 
print(r.recvline())
r.sendline(input().encode())

print(r.recvline())
mtx = identity_matrix(F, FLAG_N)
rets = []
for perm in perms:
    mtx.set_block(0, 0, perm)
    ser = serialize_mtx(mtx)
    r.sendline(ser.encode())
    ret_ = r.recvline()
    ret = deserialize_mtx(ret_.decode())
    rets.append(mtx.inverse() * ret)

print(bytes(list((rets[0] + rets[1] + rets[4] - rets[2] - rets[3]).T)[0]))

b'proof of work:\n'
b'curl -sSfL https://pwn.red/pow | sh -s s.AAATiA==./TWsHbai7liq+XAk9X8UFQ==\n'


 s.XW88LZd3gSww5RhT+55zJDOuB82/MFpX0wL0Zc6Ng4ZQDFzqJYFDYAhu63OC2netpX4Z+bw+mIb/9Yv3q7NLdrUmCwIJYabjOIBgErwLYmqKObuAu8OZx8rzHWHjaGzDGh9AVPBMR2bCPHfy0iUX3XapbftY2UZtE6c4pFYmoJBMRTKVEsRTfU+AQQUf0hg1L3zxplf68KDrqK0rC6VdcQ==


b'solution: aight, lets see what you got\n'
[561429172;1712867382;1980587061;1393696039;61625264;1789806836;978938500;505988819;1319545796;1671534629;1324431168;1132285632;1639227897;1217765588;1874010700;1033884531;348412361;74165087;1060375621;1779023752;16240267;540058091;177196162;455263494;2069452006;1819917268;803201148;1742582219;1900341296;1365835657;1404177608;1873841305;832013057;1981547538;257680081;372023987;676834513;1807165150;43649201;1546066228;1315447158;1582898370;35825509;1847150676;1541332348;1633224866;346581605;603095810;1177356550;2034899595;557223482;73797051;1101893835;1447179668;209646088]

[274522144;1293395661;2085502255;1069421419;880087363;803451543;1402430314;881980644;1550786234;1244920387;26873091;2121570106;1873019427;1454197242;886701222;236208126;1070001711;1464213420;1075840800;1724499875;87826493;607767734;589687273;1073036901;2095081600;1593682488;699083889;910495928;93662833;876269286;2078481154;1752253140;492538574;497593736;842823622;2051598224

Yes. After sleeping on it for a day I have found out that all that you need is two matrices that give identity matrix as a sum. 

So, if you have $B_1, B_2$ such that $B_1 + B_2 = I$ and you send $B_1^{-1}, B_2^{-1}$ then 

$B_1 \times \vec{e_1} + B_2 \times \vec{e_2} = B_1 \times (B_1^{-1} \times A \times B_1 \times A^{-1}) \times \vec{f} + B_2 \times (B_2^{-1} \times A \times B_2 \times A^{-1}) \times \vec{f} = A \times (B_1 + B_2) \times A^{-1} \times \vec{f} = \vec{f}$

In [7]:
from pwn import remote, context
context.log_level = "error"

host, port = "1337.sb",  20004
r = remote(host,port)

# handle pow
print(r.recvline()) 
print(r.recvline())
r.sendline(input().encode())

print(r.recvline())
M = MatrixSpace(F, FLAG_N)

m1 = M.random_element()
while m1.det() == 0:
    m1 = M.random_element()
    
m2 = identity_matrix(F, FLAG_N) - m1
rets = []
for mtx in [m1, m2]:
    ser = serialize_mtx(mtx.inverse())
    r.sendline(ser.encode())
    ret_ = r.recvline()
    ret = deserialize_mtx(ret_.decode())
    rets.append(mtx * ret)

print(bytes(list((rets[0] + rets[1]).T)[0]))

b'proof of work:\n'
b'curl -sSfL https://pwn.red/pow | sh -s s.AAATiA==.6n7u7v3TgQjyuoHaZ3toXA==\n'


 s.ARE57wVN9WEJhOtWYoInJ6fUJhd3cB8gTe11oFv82sC6btIHar7ioXQOB2jEOTqVSxFmX7Y04md5Ccw46yLhGicA2DppLuUUIDUpeNjgvwiE8k4OIl9V79xT9vDXNFYXlvcb9TvVQ0iUqZI3ywHosSmWSuFgu2amG92J9LUau5EE2dZWJOQ4/EjjeY9v3QdR78do/i9ZAPl8I/1+GlMapg==


b'solution: aight, lets see what you got\n'
[1614910421;1193150318;2124473918;839733603;161272665;146582257;581214892;178164487;1252718917;1100094847;1059309550;1846251114;383592449;2093632175;834821935;1427307673;1345600151;2048132796;545986628;1934812174;374777548;434175393;1393163375;2105645457;947908274;1361493973;1909886625;1086747122;1881598600;541634369;1388766021;1637680402;1182200686;519197514;1753023463;1696637972;1878235602;1737331445;633881946;1442994669;499801180;739183753;1639431589;1620531044;1885583714;248352805;435316002;2082017679;71469462;167548014;1841309152;1995406786;2094818147;1371707739;1551063726]

[1195049770;91226514;318109992;1461316453;769647645;1138162675;292547836;1265389763;2115422042;1217664528;541122786;1918896231;1569084064;1661035590;462302585;971872927;1992649556;2118886137;1545468753;1055266616;1671733997;135625464;329808429;1640099965;2077472027;145278748;1716543067;1164514012;2047430867;1170605626;1581738725;1559145572;2049587132;690697337;736446